# Cross Validation in Finance

In [8]:
using Dates
using MarketData
using StatsBase
# using Plots
using DataFrames
using Statistics
using MarketTechnicals
# using Lathe
using TimeSeries
using Pkg; Pkg.activate("my_MLJ_env", shared=true)
# using MLJ
# using MLDataUtils
# include("RiskLabAI-jl\Validation\PurgedKFoldCV.jl")
include("PurgedKFoldCV.jl")
using RiskLabAI

ArgumentError: ArgumentError: Package MarketTechnicals not found in current path.
- Run `import Pkg; Pkg.add("MarketTechnicals")` to install the MarketTechnicals package.

# Data
We import bitcoin data from Yahoo finance from 2021.

In [4]:
btc = yahoo("BTC-USD", YahooOpt(period1 = DateTime(2021, 1, 1), period2 = DateTime(2021, 12, 1)))["Close"]

335×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2021-01-01 to 2021-12-01
│            │ Close      │
├────────────┼────────────┤
│ 2021-01-01 │ 29374.1523 │
│ 2021-01-02 │ 32127.2676 │
│ 2021-01-03 │ 32782.0234 │
│ 2021-01-04 │ 31971.9141 │
│ 2021-01-05 │ 33992.4297 │
│ 2021-01-06 │ 36824.3633 │
│ 2021-01-07 │ 39371.043  │
│ 2021-01-08 │ 40797.6094 │
│ ⋮          │ ⋮          │
│ 2021-11-25 │ 57274.6797 │
│ 2021-11-26 │ 53569.7656 │
│ 2021-11-27 │ 54815.0781 │
│ 2021-11-28 │ 57248.457  │
│ 2021-11-29 │ 57806.5664 │
│ 2021-11-30 │ 57005.4258 │
│ 2021-12-01 │ 57229.8281 │

In [5]:
btc_df = DataFrame(btc[55:end])
btc_df[!, "Expected 55"] = ema(values(btc), 55)[1:end]
btc_df[!, "Expected 15"] = ema(values(btc), 15)[55 - 15 + 1: end]
btc_df[!, :Cross] .= 0
btc_df[!, :Cross] = ifelse.(btc_df[:, "Expected 55"] .< btc_df[:, "Expected 15"], 1, 0)
btc_ts = TimeArray(btc_df, timestamp = :timestamp)

UndefVarError: UndefVarError: ema not defined

In [ ]:
targetPositions = string.(btc_ts.Cross) # labels
data = btc_ts[:Close, Symbol("Expected 55"), Symbol("Expected 15")]
times = TimeArray(DataFrame(startTime = timestamp(btc_ts), endTime = append!(timestamp(btc_ts)[2:end], [timestamp(btc_ts)[end] + (timestamp(btc_ts)[2]-timestamp(btc_ts)[1])])), timestamp = :startTime)
train_index, test_index = Lathe.preprocess.UniformSplit(collect(1:length(btc_ts)))

# Purged Training Times
Purges test observations in the training set

In [ ]:
purgedTrainTimes(times, times[test_index])

# Embargo Times
Gets embargo time for each bar

In [ ]:
embargoTimes(timestamp(btc_ts), 0.05)

# Purged KFold and Cross Validation Score
Uses the PurgedKFold struct and functions to achieve CV score

In [ ]:
purgedKFold = PurgedKFold(5, times, 0.05)

In [ ]:
collect(purgedKFoldSplit(purgedKFold, btc_ts))

In [ ]:
tree_model = @load RandomForestClassifier pkg="DecisionTree" verbosity=0 

In [ ]:
crossValidationScore(tree_model, data, targetPositions, ones(length(data)), "Log Loss", times, purgedKFold, 5, 0.05)